# VoidKnee 🦵 -> Tradutor para C

**Por que “VoidKnee”?** homenagem (carinhosa rsrsrsrs) ao joelho do Igor Ferreira que vive pedindo férias.

## O que esse projeto mostra
1) Palavras-chave da linguagem  
2) Como usamos o tradutor  
3) Exemplos de condição, laço, IO e concatenação  
4) Limitações + próximos passos


## Dicionário VoidKnee -> C

**Tipos**
- `inteirao` -> `int`
- `flutuante` -> `float`
- `dobradura` -> `double`

**Controle**
- `sejoelho` -> `if`
- `outracoisa` -> `else`
- `praCada` -> `for`
- `enquantoDoi` -> `while`

**Entrada/Saída**
- `entradaAi(x)` -> `scanf(...)`
- `mostraAi(...)` -> `printf(...)`  (aceita `"texto" + variavel + "..."`)


## Limitações atuais (intencionais)
- Sem funções, arrays ou operadores lógicos avançados (&&, || etc).
- Foco didático: nomes em PT-BR e tradução direta pro C.
- `mostraAi` entende concatenação com `+` e escolhe `%d/%f/%lf` pelo tipo inferido.

## Próximos passos (se sobrar tempo e joelho)
- Funções com retorno (ex.: `devolve inteirao soma(...)`).
- Arrays e leitura de múltiplos valores em `entradaAi`.
- Operadores lógicos e comparação encadeada.


## Como o tradutor funciona (passo a passo)

### 1. Mapa da linguagem (VoidKnee -> C)
```python
TYPE_MAP = {
    "inteirao": "int",
    "flutuante": "float",
    "dobradura": "double",
}

CTRL_MAP = {
    "sejoelho": "if",
    "outracoisa": "else",
    "praCada": "for",
    "enquantoDoi": "while",
}

PRINTF_FMT = {"int": "%d", "float": "%f", "double": "%lf"}
SCANF_FMT  = {"int": "%d", "float": "%f", "double": "%lf"}
```

### 2. Remoção de comentários
```python

def _strip_comments(linha_codigo: str) -> str:
    caracteres_processados = []
    dentro_de_string = False
    indice_atual = 0

    while indice_atual < len(linha_codigo):
        caractere = linha_codigo[indice_atual]

        # Detecta se estamos dentro de uma string
        if caractere == '"' and (indice_atual == 0 or linha_codigo[indice_atual - 1] != '\\'):
            dentro_de_string = not dentro_de_string
            caracteres_processados.append(caractere)
            indice_atual += 1
            continue

        # Se encontrar "//" fora de string, corta o resto da linha
        if not dentro_de_string and caractere == '/' and indice_atual + 1 < len(linha_codigo) and linha_codigo[indice_atual + 1] == '/':
            break

        caracteres_processados.append(caractere)
        indice_atual += 1

    return "".join(caracteres_processados)

```

### 3. Tabela de símbolos
```python

import re
IDENT_RE = r"[A-Za-z_]\w*"

def build_symbol_table(codigo_fonte: str) -> dict[str, str]:
    """
    Cria uma tabela com as variáveis declaradas em VoidKnee
    e seus respectivos tipos em C.

    Exemplo:
    inteirao x = 0; -> {"x": "int"}
    flutuante y; -> {"y": "float"}
    """
    tabela_simbolos: dict[str, str] = {}
    expressao_declaracao = re.compile(
        rf"\b({'|'.join(TYPE_MAP.keys())})\b\s+({IDENT_RE})(?:\s*=[^;]*)?;"
    )

    for linha_original in codigo_fonte.splitlines():
        linha_sem_comentario = _strip_comments(linha_original).strip()
        correspondencia_declaracao = expressao_declaracao.search(linha_sem_comentario)
        if correspondencia_declaracao:
            tipo_vk, nome_variavel = correspondencia_declaracao.group(1), correspondencia_declaracao.group(2)
            tipo_c = TYPE_MAP[tipo_vk]
            tabela_simbolos[nome_variavel] = tipo_c
    return tabela_simbolos

```

### 4. Substituição de palavras-chave
```python

def _replace_outside_strings(texto: str, mapa_substituicao: dict[str, str]) -> str:
    """
    Faz substituição de palavras-chave fora de strings.
    """
    def substituir_palavra(correspondencia_regex):
        palavra = correspondencia_regex.group(0)
        return mapa_substituicao.get(palavra, palavra)

    padrao_regex = r"\b(" + "|".join(re.escape(chave) for chave in mapa_substituicao) + r")\b"
    caracteres_processados, dentro_de_string, indice_atual = [], False, 0

    while indice_atual < len(texto):
        caractere = texto[indice_atual]

        if caractere == '"' and (indice_atual == 0 or texto[indice_atual - 1] != '\\'):
            dentro_de_string = not dentro_de_string
            caracteres_processados.append(caractere)
            indice_atual += 1
            continue

        if not dentro_de_string:
            texto_restante = texto[indice_atual:]
            substituido = re.sub(padrao_regex, substituir_palavra, texto_restante)
            caracteres_processados.append(substituido)
            break

        caracteres_processados.append(caractere)
        indice_atual += 1

    return "".join(caracteres_processados)


def replace_keywords(linha_codigo: str) -> str:
    """
    Aplica substituição das palavras-chave VoidKnee para C.
    """
    linha_codigo = _replace_outside_strings(linha_codigo, TYPE_MAP)
    linha_codigo = _replace_outside_strings(linha_codigo, CTRL_MAP)
    return linha_codigo

```

### 5. Entrada e Saída (mostraAi e entradaAi)
```python

def parse_mostraAi_arguments(argumentos_texto: str, tabela_simbolos: dict[str, str]) -> str:
    """
    Analisa os argumentos de mostraAi e converte para printf.
    Suporta concatenação usando "+".
    """
    partes_expressao, buffer_caracteres = [], []
    dentro_de_string, indice_atual = False, 0

    # Divide os elementos por '+', respeitando strings
    while indice_atual < len(argumentos_texto):
        caractere = argumentos_texto[indice_atual]
        if caractere == '"' and (indice_atual == 0 or argumentos_texto[indice_atual - 1] != '\\'):
            dentro_de_string = not dentro_de_string
        if caractere == '+' and not dentro_de_string:
            partes_expressao.append("".join(buffer_caracteres).strip())
            buffer_caracteres = []
            indice_atual += 1
            continue
        buffer_caracteres.append(caractere)
        indice_atual += 1
    if buffer_caracteres:
        partes_expressao.append("".join(buffer_caracteres).strip())

    lista_formatos, lista_argumentos = [], []
    for parte in (p.strip() for p in partes_expressao if p.strip()):
        if parte.startswith('"') and parte.endswith('"'):
            lista_formatos.append(parte[1:-1])
        elif re.fullmatch(IDENT_RE, parte) and parte in tabela_simbolos:
            tipo_c = tabela_simbolos[parte]
            lista_formatos.append(PRINTF_FMT.get(tipo_c, "%s"))
            lista_argumentos.append(parte)
        else:
            lista_formatos.append("%g")
            lista_argumentos.append(parte)

    string_formatacao = '"' + "".join(lista_formatos) + '"'
    return f'printf({string_formatacao}{", " + ", ".join(lista_argumentos) if lista_argumentos else ""});'


def translate_io(linha_codigo: str, tabela_simbolos: dict[str, str]) -> str:
    """
    Converte comandos de IO da VoidKnee para C.
    """
    # mostraAi -> printf
    correspondencia_saida = re.search(r"\bmostraAi\b\s*\((.*)\)\s*;", linha_codigo)
    if correspondencia_saida:
        conteudo_interno = correspondencia_saida.group(1).strip()
        return re.sub(r"\bmostraAi\b\s*\(.*\)\s*;", parse_mostraAi_arguments(conteudo_interno, tabela_simbolos), linha_codigo)

    # entradaAi -> scanf
    correspondencia_entrada = re.search(r"\bentradaAi\b\s*\(\s*(" + IDENT_RE + r")\s*\)\s*;", linha_codigo)
    if correspondencia_entrada:
        nome_variavel = correspondencia_entrada.group(1)
        tipo_c = tabela_simbolos.get(nome_variavel, None)
        formato_c = SCANF_FMT.get(tipo_c or "", "%d")
        return re.sub(r"\bentradaAi\b\s*\(.*\)\s*;", f'scanf("{formato_c}", &{nome_variavel});', linha_codigo)

    return linha_codigo

```

### 7. Juntando tudo e gerando o código C
Esta é a função **translate**, que orquestra todo o processo de tradução.  
Ela recebe o código em VoidKnee, passa pelas etapas de substituição de palavras-chave, 
tradução de entrada/saída (`mostraAi`, `entradaAi`), tratamento de comentários e 
controle de indentação, produzindo como saída um programa em C completo.

```python
def translate(codigo_fonte: str) -> str:
    linhas_codigo = codigo_fonte.splitlines()
    tabela_simbolos = build_symbol_table(codigo_fonte)
    saida_c = ["#include <stdio.h>", "", "int main(void) {"]
    nivel_indentacao = 1

    for linha_original in linhas_codigo:
        linha_sem_quebra = linha_original.rstrip("\n")

        if linha_sem_quebra.strip().startswith("///"):
            saida_c.append(("    " * nivel_indentacao) + f"/* {linha_sem_quebra.strip()[3:].strip()} */")
            continue

        linha_limpa = _strip_comments(linha_sem_quebra).strip()
        if not linha_limpa:
            saida_c.append(""); continue

        if linha_limpa.count("}"):
            nivel_indentacao = max(1, nivel_indentacao - linha_limpa.count("}"))

        linha_traduzida = replace_keywords(linha_limpa)
        linha_traduzida = translate_io(linha_traduzida, tabela_simbolos)

        if not linha_traduzida.strip().endswith(("{", "}", ";")):
            linha_traduzida += ";"

        saida_c.append(("    " * nivel_indentacao) + linha_traduzida)

        if linha_limpa.count("{"):
            nivel_indentacao += linha_limpa.count("{")

    saida_c += ["    return 0;", "}"]
    return "\n".join(saida_c)

```

### 8. Execução do código C traduzido

Além de traduzir para C, também criamos uma função que compila e executa o programa 
diretamente, fechando o ciclo de VoidKnee -> C -> execução.

```python
def executar_codigo_c(codigo_c: str, nome_arquivo: str = "saida.c"):
    """
    Salva o código em C, compila com gcc e executa o binário gerado.
    """
    with open(nome_arquivo, "w", encoding="utf-8") as arquivo_c:
        arquivo_c.write(codigo_c)

    subprocess.run(["gcc", nome_arquivo, "-o", "saida_exec"], check=True)
    resultado = subprocess.run(["./saida_exec"], capture_output=True, text=True)

    print("\n=== Saída do programa em C ===")
    print(resultado.stdout)

```

In [5]:
# Exemplo 01 -> Condicional com entrada

from voidknee_translator import translate

source = """
inteirao x;
mostraAi("Digite um número: ");
entradaAi(x);

sejoelho (x > 0) {
    mostraAi("positivo");
} outracoisa {
    mostraAi("negativo");
}
"""

c_code = translate(source)
print(c_code)
with open("ex1_condicional.c", "w", encoding="utf-8") as f:
    f.write(c_code)

#include <stdio.h>

int main(void) {

    int x;
    printf("Digite um número: ");
    scanf("%d", &x);

    if (x > 0) {
        printf("positivo");
    } else {
        printf("negativo");
    }
    return 0;
}


In [ ]:
# Exemplo 02 -> Laço praCada (for) com entrada

from voidknee_translator import translate

source = """
inteirao limite;
mostraAi("Digite um limite: ");
entradaAi(limite);

inteirao tentativaAtual;
praCada (tentativaAtual = 0; tentativaAtual < limite; tentativaAtual = tentativaAtual + 1) {
    mostraAi("Tentativa: " + tentativaAtual);
}
"""

c_code = translate(source)
print(c_code)
with open("ex2_pracada.c", "w", encoding="utf-8") as f:
    f.write(c_code)


In [ ]:
# Exemplo 03 -> enquantoDoi (while) com entrada

from voidknee_translator import translate

source = """
inteirao limite;
mostraAi("Digite até onde contar: ");
entradaAi(limite);

inteirao contador = 0;
enquantoDoi (contador < limite) {
    mostraAi("Contador: " + contador);
    contador = contador + 1;
}
"""

c_code = translate(source)
print(c_code)
with open("ex3_enquantodoi.c", "w", encoding="utf-8") as f:
    f.write(c_code)

In [ ]:
# Exemplo 04 -> entrada de dados + decisão

from voidknee_translator import translate

source = """
inteirao idade;
mostraAi("Digite sua idade: ");
entradaAi(idade);

sejoelho (idade >= 18) {
    mostraAi("Maior de idade");
} outracoisa {
    mostraAi("Menor de idade");
}
"""

c_code = translate(source)
print(c_code)
with open("ex4_entrada_decisao.c", "w", encoding="utf-8") as f:
    f.write(c_code)


In [ ]:
# Exemplo 05 -> Concatenação com entrada

from voidknee_translator import translate

source = """
inteirao qtd;
flutuante desconto;
dobradura preco;

mostraAi("Digite a quantidade: ");
entradaAi(qtd);
mostraAi("Digite o desconto (0.0 ~ 1.0): ");
entradaAi(desconto);
mostraAi("Digite o preço: ");
entradaAi(preco);

mostraAi("Qtd: " + qtd + " | Desc: " + desconto + " | Preco: " + preco);
"""

c_code = translate(source)
print(c_code)
with open("ex5_concatenacao.c", "w", encoding="utf-8") as f:
    f.write(c_code)


In [ ]:
# Exemplo 06 -> Somatório com entrada

from voidknee_translator import translate

source = """
/// Somatório com entrada
inteirao limite;
mostraAi("Digite até onde somar: ");
entradaAi(limite);

inteirao soma = 0;
inteirao i;

praCada (i = 1; i <= limite; i = i + 1) {
    soma = soma + i;
}

mostraAi("Soma de 1 até " + limite + " = " + soma);
"""

c_code = translate(source)
print(c_code)
with open("ex6_somatorio.c", "w", encoding="utf-8") as f:
    f.write(c_code)